In [1]:
import time
import numpy as np
import pandas as pd

import pyvinecopulib as pv
from gcimpute.helper_evaluation import get_smae
from vcimpute.generate_data import make_complete_data_matrix, mask_MCAR
from vcimpute.zeisberger import VineCopReg
from gcimpute.gaussian_copula import GaussianCopula

In [ ]:
%%capture

n = 1000
R = 10
out = []
for d, d_mis in [(10,2), (10,3), (10,4), (10,5), (25, 5), (25, 7), (25, 9), (25, 11)]:
    for mask_fraction in [0.1, 0.2, 0.3, 0.4]:
        for copula_type in ['gaussian', 'clayton', 'frank', 'student']:
            for r in range(R):
                X = make_complete_data_matrix(n, d, copula_type, vine_structure='R')
                X_mis = mask_MCAR(X, mask_fraction=mask_fraction, d_mis=d_mis, monotonic_missingness=True)
                missingness = np.count_nonzero(np.isnan(X_mis))/(n*d)

                start = time.process_time()
                X_imp = VineCopReg([copula_type], 10, 'R').fit_transform(X_mis)
                end = time.process_time()

                smae = get_smae(x_imp=X_imp, x_true=X, x_obs=X_mis)
                smae_mean = smae[~np.isnan(smae)].mean()
                
                out.append((smae_mean, end-start, 'vcimpute', n, d, d_mis, mask_fraction, missingness, r))
                
                start = time.process_time()
                X_imp = GaussianCopula().fit_transform(X_mis)
                end = time.process_time()

                smae = get_smae(x_imp=X_imp, x_true=X, x_obs=X_mis)
                smae_mean = smae[~np.isnan(smae)].mean()
                out.append((smae_mean, end-start, 'gcimpute', n, d, d_mis, mask_fraction, missingness, r))

In [ ]:
df = pd.DataFrame(out, columns=['smae', 'elapsed', 'method', 'n', 'd', 'd_mis', 'mask_frac', 'missingness', 'rep'])

In [ ]:
1+1